In [1]:
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt

import pycountry

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go


sns.set_context("talk")
plt.style.use('ggplot')
mpl.rcParams['axes.labelpad']='20'

## datos de https://data.unicef.org/resources/dataset/sowc-2019-statistical-tables/

In [2]:
df = pd.read_excel("../datos/infant_mortality.xlsx", skipfooter=31, skiprows=5)
columnas = ["Unnamed: 1","Unnamed: 14", "Unnamed: 16"]
df = df.loc[:,columnas]
df = df.rename(columns={"Unnamed: 1": "name","Unnamed: 14":"1990", "Unnamed: 16":"2018"})
df.head(3)

,name,1990,2018
0,Afghanistan,121,48
1,Albania,35,8
2,Algeria,42,20


In [3]:
def code_name(row):
    try:
        row["alpha3"] = pycountry.countries.search_fuzzy(row["name"])[0].alpha_3
    except:
        print(row["name"])
    return row

In [4]:
df = df.apply(code_name, axis=1)
df = df.replace({'−': 0})

In [5]:
df["2018"].max()

84

In [19]:
import plotly
import plotly.io as pio
years = ["1990", "2018"]
for year in years:
    data=dict(
        type = 'choropleth',
        locations = df['alpha3'],
        z = df[year],
        text = df["name"],
        colorscale = 'Reds',
        marker_line_color='black',
        marker_line_width=0.5,
        colorbar_title = '‰',
        zauto=False,
        zmax=175,
    )

    layout = dict(geo=dict(
            showframe=False,
            showcoastlines=True,
            projection_type='equirectangular'
        ))

    fig = go.Figure(data = [data], layout = layout)

    fig.update_layout(
        title="Mortalidad infantil en " + year,
        font=dict(
            family="Courier New, monospace",
            size=28,
            color="#7f7f7f"
        )
    )
    iplot(fig)
    plotly.offline.plot(fig,image = 'png', image_filename=year) #no poner el autoopen porque para que se guarde la imagen tiene que abrirse en el navegador primero. Sino hay que instalar orca o algo asi